# [教學網站-莫凡](https://morvanzhou.github.io/tutorials/python-basic/threading/2-add-thread/)
# [官方文檔](https://docs.python.org/zh-cn/3/library/threading.html?highlight=thread#thread-local-data)
# [GTW教學](https://blog.gtwang.org/programming/python-threading-multithreaded-programming-tutorial/)

In [1]:
import threading

# 取得目前的線程數量

In [8]:
threading.active_count()

5

# 查詢目前線程資訊

In [5]:
threading.enumerate()
#輸出的結果是一個<_MainThread(...)>帶多個<Thread(...)>。

[<_MainThread(MainThread, started 52140)>,
 <Thread(Thread-4, started daemon 30592)>,
 <Heartbeat(Thread-5, started daemon 1732)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 18116)>,
 <ParentPollerWindows(Thread-3, started daemon 26468)>]

# 查詢目前正在執行的線程

In [6]:
threading.current_thread()

<_MainThread(MainThread, started 52140)>

# 添加線程，threading.Thread()接收參數target代表這個線程要完成的任務，需自行定義

In [7]:
def thread_job():
    print('This is a thread of %s' % threading.current_thread())

def main():
    thread = threading.Thread(target=thread_job,)   # 增加定義線程 
    thread.start()  # 讓線程開始工作
    
if __name__ == '__main__':
    main()

This is a thread of <Thread(Thread-6, started 22768)>


# join()功能 等待所有工作完成

## 不加 join() 的結果
### 我們讓 T1 線程工作的耗時增加

In [2]:
import threading
import time
def thread_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1) # 任務間隔0.1s
    print("T1 finish\n")

def main():
    added_thread = threading.Thread(target=thread_job, name='T1')
    added_thread.start()
    print("all done\n")
    
if __name__ == '__main__':
    main()

T1 start
all done




## 預想結果應該為
T1 start <br />
T1 finish <br />
all done <br />
## 但是實際是
T1 start <br />
all done <br />
T1 finish <br />

# 加入 join() 的結果
線程任務還未完成便輸出all done。如果要遵循順序，可以在啟動線程後對它調用join：<br />
added_thread.start() <br />
added_thread.join() <br />
print("all done\n") <br />

## 使用join對控制多個線程的執行順序非常關鍵。舉個例子，假設我們現在再加一個線程T2，T2的任務量較小，會比T1更快完成：

In [7]:
import threading
import time
def T1_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1)
    print("T1 finish\n")

def T2_job():
    print("T2 start\n")
    print("T2 finish\n")

thread_1 = threading.Thread(target=T1_job, name='T1')
thread_2 = threading.Thread(target=T2_job, name='T2')
thread_1.start() # 開啟T1
thread_2.start() # 開啟T2
print("all done\n")
#因jupyter notebook 關西 無法顯示等待跑較慢的range結果

T1 start

T2 start

T2 finish

all done



## 結果為
T1 start <br />
T2 start <br />
T2 finish <br />
all done <br />
T1 finish <br />

## 現在T1和T2都沒有join，注意這裡說”一種”是因為all done的出現完全取決於兩個線程的執行速度， 完全有可能T2 finish出現在all done之後。這種雜亂的執行方式是我們不能忍受的，因此要使用join加以控制。

## 我們試試在T1啟動後，T2啟動前加上thread_1.join():
    thread_1.start()
    thread_1.join() # notice the difference! 
    thread_2.start()
    print("all done\n")

In [5]:
import threading
import time
def T1_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1)
    print("T1 finish\n")

def T2_job():
    print("T2 start\n")
    print("T2 finish\n")

thread_1 = threading.Thread(target=T1_job, name='T1')
thread_2 = threading.Thread(target=T2_job, name='T2')
thread_1.start() # 開啟T1
thread_1.join() # notice the difference!
thread_2.start() # 開啟T2
print("all done\n")

T1 start

T1 finish

T2 start
all done


T2 finish



## 結果為
    T1 start
    T1 finish
    T2 start
    all done
    T2 finish
可以看到，T2會等待T1結束後才開始運行。

## 如果我們在T2啟動後放上thread_1.join()會怎麼樣呢？
    thread_1.start()
    thread_2.start()
    thread_1.join() # notice the difference!
    print("all done\n")

In [9]:
import threading
import time
def T1_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1)
    print("T1 finish\n")

def T2_job():
    print("T2 start\n")
    print("T2 finish\n")

thread_1 = threading.Thread(target=T1_job, name='T1')
thread_2 = threading.Thread(target=T2_job, name='T2')
thread_1.start() # 開啟T1
thread_2.start() # 開啟T2
thread_1.join() # notice the difference! 這樣T1就會跟著T2一起執行
print("all done\n")

T1 start

T2 start

T2 finish

T1 finish

all done



## 結果 
    T1 start
    T2 start
    T2 finish
    T1 finish
    all done
T2在T1之後啟動，並且因為T2任務量小會在T1之前完成；而T1也因為加了join，all done在它完成後才顯示。

## 你也可以添加thread_2.join()進行嘗試，但為了規避不必要的麻煩，推薦如下這種1221的V型排布：
    thread_1.start() # start T1
    thread_2.start() # start T2
    thread_2.join() # join for T2
    thread_1.join() # join for T1
    print("all done\n")

    """
    T1 start
    T2 start
    T2 finish
    T1 finish
    all done
    """

In [10]:
import threading
import time
def T1_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1)
    print("T1 finish\n")

def T2_job():
    print("T2 start\n")
    print("T2 finish\n")

thread_1 = threading.Thread(target=T1_job, name='T1')
thread_2 = threading.Thread(target=T2_job, name='T2')
thread_1.start() # 開啟T1
thread_2.start() # 開啟T2
thread_2.join() # join for T2
thread_1.join() # join for T1
print("all done\n")

T1 start

T2 start

T2 finish

T1 finish

all done



# 儲存進程結果 Queue
    因為threading沒有返回值
    代碼實現功能，將數據列表中的數據傳入，使用四個線程處理，將結果保存在Queue中，線程執行完後，從Queue中獲取存儲的結果

In [11]:
#導入線程,隊列的標準模塊 
import threading
import time
from queue import Queue

# 定義一個被多線程調用的函數
##### 函數的參數是一個列表l和一個隊列q，函數的功能是，對列表的每個元素進行平方計算，將結果保存在隊列中


In [12]:
def job(l,q):
    for i in range (len(l)):
        l[i] = l[i]**2
    q.put(l)   #多線程調用的函數不能用return返回值, 意思是 item 放入隊列

# 定義一個多線程函數

#### 在多線程函數中定義一個Queue，用來保存返回值，代替return，定義一個多線程列表，初始化一個多維數據列表，用來處理：

In [15]:
def multithreading():
    q =Queue()    #q中存放返回值，代替return的返回值
    threads = []
    data = [[1,2,3],[3,4,5],[4,4,4],[5,5,5]]

#### 在多線程函數中定義四個線程，啟動線程，將每個線程添加到多線程的列表中

In [ ]:
for i in range(4):   #定義四個線程
    t = threading.Thread(target=job,args=(data[i],q)) #Thread首字母要大寫，被調用的job函數沒有括號，只是一個索引，參數在後面
    t.start()#開始線程
    threads.append(t) #把每個線程append到線程列表中

#### 分別join四個線程到主線程

In [ ]:
for thread in threads:
    thread.join()

#### 定義一個空的列表results，將四個線運行後保存在隊列中的結果返回給空列表results

In [ ]:
results = []
for _ in range(4):
    results.append(q.get())  #q.get()按順序從q中拿出一個值
print(results)

#### 完整代碼

In [ ]:
import threading
import time

from queue import Queue

def job(l,q):
    for i in range (len(l)):
        l[i] = l[i]**2  #2次方
    q.put(l)

def multithreading():
    q =Queue()
    threads = []
    data = [[1,2,3],[3,4,5],[4,4,4],[5,5,5]]
    for i in range(4):
        t = threading.Thread(target=job,args=(data[i],q))
        t.start()
        threads.append(t)
    for thread in threads:
        thread.join()
    results = []
    for _ in range(4):
        results.append(q.get())
    print(results)

if __name___=='__main__':
    multithreading()

#### 結果

In [ ]:
[[1, 4, 9], [9, 16, 25], [16, 16, 16], [25, 25, 25]]

In [24]:
f = lambda a,b,c:a if a > b else c if c > a else 0

In [27]:
f(1,10,800)

800